### 데이터 취득

In [ ]:
import numpy as np 
import pandas as pd

# df = pd.read_csv('https://raw.githubusercontent.com/songys/Chatbot_data/refs/heads/master/ChatbotData.csv')
# df = df[['Q', 'A']]
# df

# df.to_csv("output.txt", sep="\t", index=False)


In [8]:
df_2 = open('./output.txt', 'r', encoding='utf-8')

In [50]:
# df['A']
# a = []

# for line in df['A']:           
#     a.append(line.strip())  

# print(a[:5])  # 앞 5줄 확인


### 데이터 전처리

##### 토커나이저 학습 (sentencepiece 활용)
- 접두사, 접미사 처리 (문장의 처음과 끝에 bos, eos)
    - Train() 인자 cmd 옵션을 추가 (<bos>, <eos>, <pad>, <oov>, ...)
    - set_encode_extra_options(':') <br>
        set_encode_extra_options('bos:') <br>
        set_encode_extra_options(':eos') <br>
        set_encode_extra_options('bos:eos') <br>

In [10]:
inputs = []
kor_inputs = []
kor_targets = []

with open('output.txt', 'r', encoding='utf-8') as f :
    for line in f :
        Q, A = line.split('\t')

        kor_input = '<sos> ' + A
        kor_target = A + ' <eos>'

        inputs.append(Q)
        kor_inputs.append(kor_input)
        kor_targets.append(kor_target)

len(inputs), len(kor_inputs), len(kor_targets)


(11824, 11824, 11824)

In [11]:
inputs

['Q',
 '12시 땡!',
 '1지망 학교 떨어졌어',
 '3박4일 놀러가고 싶다',
 '3박4일 정도 놀러가고 싶다',
 'PPL 심하네',
 'SD카드 망가졌어',
 'SD카드 안돼',
 'SNS 맞팔 왜 안하지ㅠㅠ',
 'SNS 시간낭비인 거 아는데 매일 하는 중',
 'SNS 시간낭비인데 자꾸 보게됨',
 'SNS보면 나만 빼고 다 행복해보여',
 '가끔 궁금해',
 '가끔 뭐하는지 궁금해',
 '가끔은 혼자인게 좋다',
 '가난한 자의 설움',
 '가만 있어도 땀난다',
 '가상화폐 쫄딱 망함',
 '가스불 켜고 나갔어',
 '가스불 켜놓고 나온거 같아',
 '가스비 너무 많이 나왔다.',
 '가스비 비싼데 감기 걸리겠어',
 '가스비 장난 아님',
 '가장 확실한 건 뭘까?',
 '가족 여행 가기로 했어',
 '가족 여행 고고',
 '가족 여행 어디로 가지?',
 '가족 있어?',
 '가족관계 알려 줘',
 '가족끼리 여행간다.',
 '가족들 보고 싶어',
 '가족들이랑 서먹해',
 '가족들이랑 서먹해졌어',
 '가족들이랑 어디 가지?',
 '가족들이랑 여행 갈거야',
 '가족여행 가야지',
 '가족이 누구야?',
 '가족이랑 여행 가려고',
 '가족한테 스트레스 풀었어',
 '가출할까?',
 '가출해도 갈 데가 없어',
 '간만에 떨리니까 좋더라',
 '간만에 쇼핑 중',
 '간만에 휴식 중',
 '간식 뭐 먹을까',
 '간식 추천',
 '간장치킨 시켜야지',
 '간접흡연 싫어',
 '갈까 말까 고민 돼',
 '갈까 말까?',
 '감 말랭이 먹고 싶다.',
 '감 말랭이 먹어야지',
 '감기 같애',
 '감기 걸린 것 같아',
 '감기 기운이 있어',
 '감기 들 거 같애',
 '감기가 오려나',
 '감기약이 없어',
 '감기인거 같애',
 '감미로운 목소리 좋아',
 '감정이 쓰레기통처럼 엉망진창이야',
 '감정컨트롤을 못하겠어',
 '감정컨트롤이 안돼',
 '감히 나를 무시하는 애가 있어',
 '갑자기 나쁜 생각이 막 들더라',
 '갑자기 

In [12]:
# kor = []
# kor_inputs = []
# kor_targets = []


# for line in df['Q'] :
#     kor.append(line)

# for line in df['A'] :
#     kor_input = '<bos> ' + line
#     kor_target = line + ' <eos>'
#     kor_inputs.append(kor_input)
#     kor_targets.append(kor_target)

# len(kor), len(kor_inputs), len(kor_targets)
# print(kor[:10])
# print(kor_inputs[:10])
# print(kor_targets[:10])

##### 학습용 데이터 Q_input, A_input, A_target 생성 
- 시퀀싱

In [13]:
VOCAB_SIZE = 10000

In [14]:
# 질문 토큰화 
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=VOCAB_SIZE,  oov_token='<oov>', filters='')
tokenizer.fit_on_texts(inputs)

inputs_seq = tokenizer.texts_to_sequences(inputs)

inputs_seq[2500:2505]

[[3288, 3289, 2004, 328],
 [1560, 6251, 8],
 [1560, 282],
 [1560, 6252, 6253],
 [1560, 2080]]

In [15]:
for seq in inputs_seq[2500:2505] :
    print([tokenizer.index_word[idx] for idx in seq])

['셀프', '신행준비', '팁', '알려줘']
['셀프웨딩', '어떨꺼', '같아']
['셀프웨딩', '해볼까']
['셀프웨딩', '힘든만큼', '보람될까?']
['셀프웨딩', '힘들겠지']


In [16]:
tokenizer.index_word

{1: '<oov>',
 2: '너무',
 3: '좋아하는',
 4: '거',
 5: '안',
 6: '싶어',
 7: '나',
 8: '같아',
 9: '사람',
 10: '좀',
 11: '내가',
 12: '어떻게',
 13: '썸',
 14: '왜',
 15: '내',
 16: '사람이',
 17: '것',
 18: '잘',
 19: '수',
 20: '이별',
 21: '오늘',
 22: '자꾸',
 23: '이제',
 24: '없어',
 25: '또',
 26: '싶다',
 27: '많이',
 28: '다',
 29: '헤어진지',
 30: '다시',
 31: '여자친구가',
 32: '남자친구가',
 33: '더',
 34: '진짜',
 35: '게',
 36: '정말',
 37: '나를',
 38: '뭐',
 39: '하고',
 40: '하는',
 41: '있어',
 42: '있는',
 43: '계속',
 44: '있을까?',
 45: '이',
 46: '나만',
 47: '먹고',
 48: '이렇게',
 49: '연락',
 50: '못',
 51: '연애',
 52: '좋아',
 53: '있는데',
 54: '혼자',
 55: '날',
 56: '될까?',
 57: '다른',
 58: '타는',
 59: '한',
 60: '방법',
 61: '그',
 62: '없는',
 63: '그냥',
 64: '짝남이',
 65: '싶다.',
 66: '같이',
 67: '해',
 68: '나한테',
 69: '선물',
 70: '힘드네',
 71: '친구가',
 72: '같은데',
 73: '마음이',
 74: '짝사랑',
 75: '가고',
 76: '헤어진',
 77: '연락이',
 78: '줄',
 79: '술',
 80: '같아.',
 81: '좋은',
 82: '사랑',
 83: '짝남',
 84: '보고',
 85: '싫어',
 86: '해도',
 87: '할',
 88: '나는',
 89: '후',
 90: '사랑이',
 91: '짝녀가',


In [17]:
# 입력 text 단어 - 벡터 값 형태로 변환
import io

# index -> word 매핑 딕셔너리
index2word = tokenizer.index_word  

input_file = "vec_output.txt"
output_file = "vec_output_words.txt"

with open(input_file, "r", encoding="utf-8") as fin, \
     open(output_file, "w", encoding="utf-8") as fout:
    
    for line in fin:
        parts = line.strip().split()
        if not parts:
            continue
        idx = int(parts[0])              # 첫 열: 인덱스
        word = index2word.get(idx, f"<unk_{idx}>")  # 없는 경우 <unk_x>
        vector = " ".join(parts[1:])     # 나머지는 벡터값
        fout.write(f"{word} {vector}\n")


In [18]:
# 모델 학습 시 사용할 변수 생성
num_words = min(VOCAB_SIZE, len(tokenizer.word_index))
max_len = max([len(seq) for seq in inputs_seq])

In [19]:
len(tokenizer.index_word)

14284

In [20]:
# 대답
kor_tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token='<oov>', filters='')  
kor_tokenizer.fit_on_texts(kor_inputs + kor_targets)

kor_inputs_seq = kor_tokenizer.texts_to_sequences(kor_inputs)
kor_targets_seq = kor_tokenizer.texts_to_sequences(kor_targets)

In [21]:
kor_tokenizer.index_word

{1: '<oov>',
 2: '<sos>',
 3: '<eos>',
 4: '거예요.\n',
 5: '수',
 6: '더',
 7: '같아요.\n',
 8: '좋은',
 9: '잘',
 10: '있을',
 11: '있어요.\n',
 12: '많이',
 13: '것',
 14: '게',
 15: '마세요.\n',
 16: '해보세요.\n',
 17: '건',
 18: '수도',
 19: '거',
 20: '마음이',
 21: '좋을',
 22: '시간이',
 23: '그',
 24: '없어요.\n',
 25: '안',
 26: '될',
 27: '것도',
 28: '다른',
 29: '바랄게요.\n',
 30: '사람',
 31: '좀',
 32: '저도',
 33: '돼요.\n',
 34: '있는',
 35: '그런',
 36: '너무',
 37: '하세요.\n',
 38: '좋아요.\n',
 39: '보세요.\n',
 40: '제가',
 41: '바랍니다.\n',
 42: '그럴',
 43: '할',
 44: '않아요.\n',
 45: '걸',
 46: '드세요.\n',
 47: '같이',
 48: '좋죠.\n',
 49: '사람이',
 50: '해요.\n',
 51: '생각해요.\n',
 52: '좋겠어요.\n',
 53: '말해보세요.\n',
 54: '마음을',
 55: '먼저',
 56: '이제',
 57: '하고',
 58: '일이',
 59: '몰라요.\n',
 60: '자신을',
 61: '다',
 62: '하는',
 63: '생각해보세요.\n',
 64: '정말',
 65: '좋겠네요.\n',
 66: '힘든',
 67: '다시',
 68: '마음',
 69: '있으면',
 70: '아니에요.\n',
 71: '때',
 72: '한',
 73: '바라요.\n',
 74: '그게',
 75: '많은',
 76: '주세요.\n',
 77: '사랑은',
 78: '찾아보세요.\n',
 79: '말고',
 80: '만큼',
 81: '가보세요.\n'

In [22]:
kor_num_words = min(VOCAB_SIZE, len(kor_tokenizer.index_word))
kor_max_len = max([len(seq) for seq in kor_inputs_seq])

##### 패딩 

In [23]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

inputs_padded = pad_sequences(inputs_seq, maxlen=max_len, padding='pre')
kor_inputs_padded = pad_sequences(kor_inputs_seq, maxlen=kor_max_len, padding='post')
kor_targets_padded = pad_sequences(kor_targets_seq, maxlen=kor_max_len, padding='post')

In [24]:
print(inputs_padded[:10])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
  4752]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0 4753
  4754]
 [   0    0    0    0    0    0    0    0    0    0    0    0 4755  341
   543]
 [   0    0    0    0    0    0    0    0    0    0    0    0 2583  793
    26]
 [   0    0    0    0    0    0    0    0    0    0    0 2583  794  793
    26]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0 4756
  2584]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0 2585
  4757]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0 2585
    92]
 [   0    0    0    0    0    0    0    0    0    0    0  795 4758   14
  4759]
 [   0    0    0    0    0    0    0    0  795 4760    4  544  232   40
   108]]


### 모델 생성 및 학습

In [25]:
# 4개의 임베딩 된 벡터 값 반환(just 확인)
with open('./vec_output_words.txt', 'r', encoding='utf-8') as f:
    for i, vects in enumerate(f) :
        print(vects)
        if i == 3:
            break

<oov> -0.28306955 0.37589562 0.094337866 -0.05433331 0.029382184 -0.5702773 -0.06988839 0.65768623 -0.010133286 -0.3993969 -0.08205416 -0.24057753 -0.0040303823 0.011710708 0.08503822 -0.24766243 0.33792344 -0.5333098 0.044761464 -0.7518131 0.13501486 0.26749036 0.31715935 -0.056275327 -0.09621481 -0.09927533 -0.21606618 -0.2719294 -0.35155487 0.0060534864 0.5321063 0.15298057 0.07474075 -0.41104507 0.1345209 0.48215735 -0.024528481 -0.19637153 -0.19106258 -0.37253225 0.31304148 -0.33402684 -0.17817016 -0.029223101 0.41250378 -0.1301909 -0.28776067 -0.06778694 0.03760286 0.2629352 0.18452084 -0.20442182 -0.20635007 0.108239904 -0.23415731 0.13708046 0.1183795 0.072153494 -0.35990456 -0.16938588 0.047179505 0.0754158 0.16412306 0.06487503 -0.2340602 0.4694691 -0.050618738 0.14589398 -0.25276604 0.3250335 -0.050467398 0.12320346 0.1513699 -0.054996453 0.20874794 0.061510373 0.11757421 0.067297675 -0.0762281 0.09435649 -0.3334876 -0.18223114 -0.16455485 0.28382778 0.03606505 -0.090989105 

In [26]:
import numpy as np

# 임베딩 벡터 매트릭스 확인
def make_embedding_matrix(num_words, embedding_dim, tokenizer, file_path) :
    embedding_matrix = np.zeros((num_words + 1, embedding_dim))

    pretrained_embedding = {}

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f :
            word, *vects = line.split()                # 공백기준으로 단어, 벡터 리스트 분리
            vects = np.array(vects, dtype=np.float32)  # 리스트 형식으로 반환
            pretrained_embedding[word] = vects         # 단어와 벡터가 맵핑된 딕셔너리 형태로 반환 (각 리스트를 '단어'를 키로 반환)

    for word, index in tokenizer.word_index.items():
        vects_ = pretrained_embedding.get(word)
        if vects_ is not None :
            embedding_matrix[index] = vects_           # 각 토큰화된 토큰을 정수 인코딩

    return embedding_matrix                            # 정수 임베딩된 단어 + 벡터 임베딩된 단어

In [27]:
EMBEDDING_DIM = 100

embedding_matrix = make_embedding_matrix(
    num_words,
    EMBEDDING_DIM,
    tokenizer,
    'vec_output_words.txt'
)

embedding_matrix.shape

(10001, 100)

In [28]:
word_index = ['<ped>'] + list(tokenizer.index_word.values())

In [29]:
# import pandas as pd

# pd.DataFrame(embedding_matrix, index = word_index)

##### 인코더 생성  

In [30]:
from tensorflow.keras import layers, models

LATENT_DIM = 512

encoder_inputs = layers.Input(shape=(max_len, ))
en_embedding_layer = layers.Embedding(num_words + 1, EMBEDDING_DIM, weights=[embedding_matrix]) # eng_num_words + 1 : 전체 길이 + 패딩토큰, weights=[en_embedding_matrix] : 가중치를 사전 학습된 임베딩 벡터로
x = en_embedding_layer(encoder_inputs)
encoder_outputs, h, c = layers.LSTM(LATENT_DIM, return_state=True)(x)  # 은닉층 차원은 512로 , 마지막 상태에 대한 반환값으로 결과
encoder_states = [h, c]  # 번역을 실시하기 위해서는 맥락과 흐름을 알아야 하기 때문에 h, c 를 마지막 아웃풋으로 반환

encoder_model = models.Model(inputs=encoder_inputs, outputs=encoder_states)
encoder_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 15)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 100)        │     1,000,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ [(None, 512), (None,   │     1,255,424 │
│                                 │ 512), (None, 512)]     │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,255,524 (8.60 MB)

 Trainable params: 2,255,524 (8.60 MB)

 Non-trainable params: 0 (0.00 B)

##### 디코더 생성(teacher-forcing 모델) 생성 : keras

In [31]:
decoder_inputs = layers.Input(shape=(kor_max_len,))
ko_embedding_layer = layers.Embedding(kor_num_words+1, EMBEDDING_DIM)
x = ko_embedding_layer(decoder_inputs)                                            # 임베딩 통과한 인풋 값

decoder_lstm = layers.LSTM(LATENT_DIM, return_sequences=True, return_state=True)  # lstm 레이어 : 각각의 시퀀스 마다 나오는 출력값과 상태값을 가짐
x, h, c = decoder_lstm(x, initial_state=encoder_states)                           # 초기의 스테이트 값을 인코더에서 넘어온 값을 사용.

decoder_dense = layers.Dense(kor_num_words + 1, activation='softmax')             # 각 레이어에서 확률상으로 반환될 값 계산 ????
decoder_outputs = decoder_dense(x)

decoder_teacher_forcing_model = models.Model(
    inputs = [encoder_inputs, decoder_inputs],
    outputs = decoder_outputs
)

decoder_teacher_forcing_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 22)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 15, 100)   │  1,000,100 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 22, 100)   │  1,000,100 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 512),     │  1,255,424 │ embedding[0][0]   │
│                     │ (None, 512),      │            │                   │
│                     │ (None, 512)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 22, 512), │  1,255,424 │ embedding_1[0][0… │
│                     │ (None, 512),      │            │ lstm[0][1],       │
│                     │ (None, 512)]      │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 22, 10001) │  5,130,513 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 9,641,561 (36.78 MB)

 Trainable params: 9,641,561 (36.78 MB)

 Non-trainable params: 0 (0.00 B)

##### 학습

In [32]:
decoder_teacher_forcing_model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

history = decoder_teacher_forcing_model.fit(
    [inputs_padded, kor_inputs_padded],
    kor_targets_padded,
    batch_size = 64,
    epochs = 70,
    validation_split = 0.2
)

Epoch 1/70
148/148 ━━━━━━━━━━━━━━━━━━━━ 64s 421ms/step - accuracy: 0.8178 - loss: 1.8399 - val_accuracy: 0.8042 - val_loss: 1.7991
Epoch 2/70
148/148 ━━━━━━━━━━━━━━━━━━━━ 62s 419ms/step - accuracy: 0.8398 - loss: 1.3135 - val_accuracy: 0.8045 - val_loss: 1.7838
Epoch 3/70
148/148 ━━━━━━━━━━━━━━━━━━━━ 62s 418ms/step - accuracy: 0.8400 - loss: 1.2503 - val_accuracy: 0.8049 - val_loss: 1.7824
Epoch 4/70
148/148 ━━━━━━━━━━━━━━━━━━━━ 62s 419ms/step - accuracy: 0.8406 - loss: 1.2057 - val_accuracy: 0.8053 - val_loss: 1.8039
Epoch 5/70
148/148 ━━━━━━━━━━━━━━━━━━━━ 64s 430ms/step - accuracy: 0.8428 - loss: 1.1706 - val_accuracy: 0.8016 - val_loss: 1.8327
Epoch 6/70
148/148 ━━━━━━━━━━━━━━━━━━━━ 64s 432ms/step - accuracy: 0.8445 - loss: 1.1377 - val_accuracy: 0.8009 - val_loss: 1.8576
Epoch 7/70
148/148 ━━━━━━━━━━━━━━━━━━━━ 64s 432ms/step - accuracy: 0.8460 - loss: 1.1048 - val_accuracy: 0.8004 - val_loss: 1.8941
Epoch 8/70
148/148 ━━━━━━━━━━━━━━━━━━━━ 62s 418ms/step - accuracy: 0.8473 - loss: 1

In [47]:
# 모델 저장
decoder_teacher_forcing_model.save('qna_model.keras')

In [48]:
# 모델 로드
from tensorflow.keras.models import load_model

decoder_teacher_forcing_model = load_model('qna_model.keras')

### 모델 추론

##### 디코더 (추론모델) 생성

In [40]:
decoder_hidden_state = layers.Input(shape=(LATENT_DIM,))
decoder_cell_state = layers.Input(shape=(LATENT_DIM,))
decoder_state_inputs = [decoder_hidden_state, decoder_cell_state]

decoder_single_input = layers.Input(shape=(1, ))

x = ko_embedding_layer(decoder_single_input)

x, h, c = decoder_lstm(x, initial_state=decoder_state_inputs)
decoder_states = [h, c]

decoder_outputs_ = decoder_dense(x)

decoder_inference_model = models.Model(
    inputs = [decoder_single_input] + decoder_state_inputs,
    outputs = [decoder_outputs_] + decoder_states
)

decoder_inference_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 1, 100)    │  1,000,100 │ input_layer_7[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_5       │ (None, 512)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_6       │ (None, 512)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 1, 512),  │  1,255,424 │ embedding_1[2][0… │
│                     │ (None, 512),      │            │ input_layer_5[0]… │
│                     │ (None, 512)]      │            │ input_layer_6[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1, 10001)  │  5,130,513 │ lstm_1[2][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7,386,037 (28.18 MB)

 Trainable params: 7,386,037 (28.18 MB)

 Non-trainable params: 0 (0.00 B)

##### 추론 함수 

In [41]:
def translate(input_seq) :
  encoder_states_value = encoder_model.predict(input_seq)
  decoder_states_value = encoder_states_value

  sos_index = kor_tokenizer.word_index['<sos>']
  eos_index = kor_tokenizer.word_index['<eos>']

  target_seq = np.zeros((1, 1))
  target_seq[0, 0] = sos_index

  output_sentence = []

  for _ in range(kor_max_len) :
    output_tokens, h, c = decoder_inference_model.predict([target_seq] + decoder_states_value)

    pred_proba = output_tokens[0, 0, :]     # 인덱싱
    pred_index = np.argmax(pred_proba)      # 가장 높은 확률값을 뽑는 출력값 뽑기

    if pred_index == eos_index :
      break

    if pred_index > 0 :                     # 패딩 토큰 제외
      word = kor_tokenizer.index_word[pred_index]
      output_sentence.append(word)

    target_seq[0, 0] = pred_index           # 현재의 출력으로 바꾸기
    decoder_states_value = [h, c]           # 현재 state 의 값으로 변환

  return " ".join(output_sentence)

In [35]:
print(inputs[:1])  # 패딩 처리까지 해준 첫번째 값 반환

input_seq = inputs_padded[:1]
translate(input_seq)

['Q']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


'a\n'

In [36]:
for _ in range(5) :
  idx = np.random.choice(len(inputs_padded))
  input_seq = inputs_padded[idx:idx + 1]
  output_sent = translate(input_seq)
  display("입력 : ", inputs[idx])
  display("학습한 국문 : ", kor_inputs[idx])
  display("추론한 국문 : ", output_sent)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


'입력 : '

'보이스피싱 같은데'

'학습한 국문 : '

'<sos> 경찰에 신고하고 취할 수 있는 조취를 취해보세요.\n'

...

'서로 좋아하는 사람 만나기가 정말 힘든거였네.'

'학습한 국문 : '

'<sos> 쉽지 않은 일이죠.\n'

'추론한 국문 : '

'사랑은 변하는 거예요.\n'

In [37]:
def translate_eng2kor(test_text) :
  input_seq = tokenizer.texts_to_sequences(test_text)
  input_seq = pad_sequences(input_seq, maxlen=max_len)
  return translate(input_seq)

test_texts = [
    "오늘 저녁 뭐 먹을까?",
    "연락처를 물어보면 어때?",
    "아침 먹고 출근했어.",
    "무슨 차를 사야하지?",
    "저녁 메뉴 추천해줘."
]

for test_text in test_texts :
  kor_text = translate_eng2kor([test_text])
  display(test_text, kor_text)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


'오늘 저녁 뭐 먹을까?'

'맛있는 거 드세요.\n'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


'연락처를 물어보면 어때?'

...

'무슨 차를 사야하지?'

'다행이네요.\n'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


'저녁 메뉴 추천해줘.'

'엄청 놀랐겠네요.\n'

##### 테스트

### 간단한 Chatbot 구현 
1. 사용자의 입력을 받아 (처리)
2. 추론 함수에 전달 
3. 응답을 출력
4. 1~3 '종료' 전까지 반복

In [45]:
import random 
from gensim.models import FastText
from itertools import count

def chatbot_def() :
    for _ in count() :
        sentence = input("하고 싶은 말을 입력하시오 (종료하고 싶다면 종료 입력): ")
        
        if sentence == '종료' :
            print('쳇봇을 종료합니다.')
            break

        else :
            kor_text = translate_eng2kor([sentence])
            display(sentence, kor_text)
            continue
            


In [46]:
chatbot_def()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


'오늘 저녁 뭐 먹을까?'

'맛있는 거 드세요.\n'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


'전화번호를 물어봐도 될까?'

'푹 쉬세요.\n'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


'왜 그렇게 말해?'

'먼저 생활패턴을 살펴 보세요.\n'

쳇봇을 종료합니다.
